In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

In [2]:
# data_train = spi.loadmat('GP_matlab.mat')

# isamp = 3500
# t = np.arange(0,1,0.01)
# u = data_train['u'][0:isamp,:]
# s1 = data_train['s1'][:,0:isamp].T

# pps = 20
# tsamp = pps*isamp

# pointer = np.random.randint(0,100,tsamp)

# u_in = np.zeros([tsamp,100])
# x_t_in = np.zeros([tsamp,1])
# s_in = np.zeros([tsamp,1])
# for i in range(0,isamp):
#     for j in range(0,pps):
#         u_in[i*20+j,:] = u[i,:]
#         x_t_in[i*20+j,:] = t[pointer[i*20+j]]
#         s_in[i*20+j,:] = s1[i,pointer[i*20+j]]
        
# spi.savemat('GP_train_data.mat',{'u_in':u_in, 'x_t_in':x_t_in, 's_in':s_in})
                
# t.shape, u.shape, s1.shape, u_in.shape, x_t_in.shape, s_in.shape

isamp = 3500
t = np.arange(0,1,0.01)

data_train = spi.loadmat('GP_train_data.mat')

u_in = data_train['u_in']
x_t_in = data_train['x_t_in']
s_in = data_train['s_in']

u_in.shape, x_t_in.shape, s_in.shape

((70000, 100), (70000, 1), (70000, 1))

In [3]:
max_u = np.max(u_in)
min_u = np.min(u_in)

u_in = (u_in-min_u)/(max_u-min_u)

max_t = np.max(x_t_in)
min_t = np.min(x_t_in)

x_t_in = (x_t_in-min_t)/(max_t-min_t)

max_s = np.max(s_in)
min_s = np.min(s_in)

s_in = (s_in-min_s)/(max_s-min_s)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 70000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

tfd = tfp.distributions
tfb = tfp.bijectors

def normal_sp(params):
    return tfd.Normal(loc = params[:, 0:1], scale = 0.001+tf.math.softplus(params[:, 1:2]))    

def negloglikelihood(y_true, y_pred):
    return tf.keras.backend.sum(-y_pred.log_prob(y_true))+(sum(model.losses)/bs)

hln = 25

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (1,), name = 'inputsT')
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = tfp.layers.DenseFlipout(2)(combined)

dist = tfp.layers.DistributionLambda(normal_sp)(output)
model = Model(inputs = [inputsB, inputsT], outputs = dist)    

model.summary()

/home/user/anaconda3/envs/SG_Env_TFP/lib/python3.9/site-packages/tensorflow_probability/python/layers/util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  loc = add_variable_fn(
2022-10-26 11:22:15.907796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-26 11:22:15.907873: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-26 11:22:15.908647: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is opt

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense_flipout (DenseFlipout)   (None, 25)           5025        ['inputsB[0][0]']                
                                                                                                  
 dense_flipout_4 (DenseFlipout)  (None, 25)          75          ['inputsT[0][0]']                
                                                                                              

2022-10-26 11:22:16.741529: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
spe = 25
string = './model/modelC2_S1_final/'+str(spe)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss_value = 0
        for i in range(0,spe):        
            logits = model({"inputsB":u_in, "inputsT":x_t_in}, training=True)
            loss_value = loss_value + negloglikelihood(s_in, logits)
        loss_value = loss_value*(1/spe)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

epochs = 10000
loss = np.zeros(epochs)

for epoch in range(epochs):
    loss_value = train_step()
    loss[epoch] = loss_value.numpy()
    if loss[epoch] <= np.min(loss[0:epoch+1]):
        model.save_weights(string)
        last_saved_wt = epoch
    if epoch%10 == 0:
        print("Epoch %d, loss %.2f" % (epoch, loss[epoch]))

print(last_saved_wt)

Epoch 0, loss 62904.19
Epoch 10, loss 61828.45
Epoch 20, loss 60882.49
Epoch 30, loss 60010.36
Epoch 40, loss 59150.02
Epoch 50, loss 58288.00
Epoch 60, loss 57424.05
Epoch 70, loss 56557.66
Epoch 80, loss 55688.90
Epoch 90, loss 54817.91
Epoch 100, loss 53944.81
Epoch 110, loss 53069.77
Epoch 120, loss 52192.95
Epoch 130, loss 51314.56
Epoch 140, loss 50434.81
Epoch 150, loss 49553.90
Epoch 160, loss 48672.08
Epoch 170, loss 47789.59
Epoch 180, loss 46906.69
Epoch 190, loss 46023.64
Epoch 200, loss 45140.72
Epoch 210, loss 44258.21
Epoch 220, loss 43376.41
Epoch 230, loss 42495.62
Epoch 240, loss 41616.15
Epoch 250, loss 40738.33
Epoch 260, loss 39862.44
Epoch 270, loss 38988.83
Epoch 280, loss 38117.83
Epoch 290, loss 37249.75
Epoch 300, loss 36384.91
Epoch 310, loss 35523.66
Epoch 320, loss 34666.30
Epoch 330, loss 33813.16
Epoch 340, loss 32964.54
Epoch 350, loss 32120.73
Epoch 360, loss 31282.03
Epoch 370, loss 30448.72
Epoch 380, loss 29621.04
Epoch 390, loss 28799.28
Epoch 400, 

In [7]:
# string = './model/modelC2_S1_final'
# model.load_weights(string)

In [ ]:
# # data_train = spi.loadmat('GP_matlab.mat')

# # t = np.arange(0,1,0.01)
# # u = data_train['u'][10000:20000,:]
# # s1 = data_train['s1'][:,10000:20000].T

# # pointer = np.random.randint(0,100,10000)

# # u_in_test = np.zeros([10000,100])
# # x_t_in_test = np.zeros([10000,1])
# # s_in_test = np.zeros([10000,1])
# # for i in range(0,10000):
# #     u_in_test[i,:] = u[i,:]
# #     x_t_in_test[i,:] = t[pointer[i]]
# #     s_in_test[i,:] = s1[i,pointer[i]]
    
# # spi.savemat('GP_test_data.mat',{'u_in_test':u_in_test, 'x_t_in_test':x_t_in_test, 's_in_test':s_in_test})
        
# # t.shape, u.shape, s1.shape, u_in_test.shape, x_t_in_test.shape, s_in_test.shape

# t = np.arange(0,1,0.01)
# data_train = spi.loadmat('GP_test_data.mat')

# u_in_test = data_train['u_in_test']
# x_t_in_test = data_train['x_t_in_test']
# s_in_test = data_train['s_in_test']

# u_in_test.shape, x_t_in_test.shape, s_in_test.shape

In [ ]:
# u_in_test = (u_in_test-min_u)/(max_u-min_u)
# x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
# s_in_test = (s_in_test-min_s)/(max_s-min_s)

In [ ]:
# nsamples = 100
# nps = 1
# pred = np.zeros([nsamples,10000])
# for i in range(0,nsamples):
#     if i%5 == 0:
#         print(i)
#     pred[i,:] = np.squeeze((model({"inputsB":u_in_test, "inputsT":x_t_in_test})).sample(1))
    
# print()
# print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2))
# print(np.mean((s_in_test)**2))
# print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2)/np.mean((s_in_test)**2))    

In [8]:
model.save_weights('./model/modelC2_S1_final')

In [9]:
data_train = spi.loadmat('GP_matlab.mat')

t = np.arange(0,1,0.01)
u = data_train['u'][10000:20000,:]
s = data_train['s1'][:,10000:20000].T

u_in_test = np.zeros([1000000,100])
x_t_in_test = np.zeros([1000000,1])
s_in_test = np.zeros([1000000,1])
for i in range(0,10000):
    for j in range(0,100):
        u_in_test[100*i+j,:] = u[i,:]
        x_t_in_test[100*i+j,:] = t[j]
        s_in_test[100*i+j,:] = s[i,j]

In [10]:
u_in_test = (u_in_test-min_u)/(max_u-min_u)
x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
s_in_test = (s_in_test-min_s)/(max_s-min_s)

In [11]:
nsamples = 100
nps = 1
pred = np.zeros([nsamples*nps,1000000])
for i in range(0,nsamples):
    if i%5 == 0:
        print(i)
    pred[nps*i:nps*(i+1),:] = np.squeeze((model({"inputsB":u_in_test, "inputsT":x_t_in_test})).sample(nps))

pred = (pred*(max_s-min_s))+min_s 
s_in_test = (s_in_test*(max_s-min_s))+min_s

print()
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2)/np.mean((s_in_test)**2))    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95

8.542363840678606e-06
0.0380138393414362
0.00022471720796081703


In [ ]:
# string = './CII_PDFs.mat'
# data_test = spi.savemat(string,{'pred':pred, 's_in_test':s_in_test})

In [ ]:
# data_train = spi.loadmat('GP_matlab.mat')

# t = np.arange(0,1,0.01)
# u = data_train['u'][isamp:10000+isamp,:]
# s1 = data_train['s1'][:,isamp:10000+isamp].T

# samp = 500

# u_in_ts = np.zeros([100,100])
# x_t_in_ts = np.zeros([100,1])
# s_in_ts = np.zeros([100,1])

# for i in range(0,100):
#     u_in_ts[i,:] = u[samp,:]
#     x_t_in_ts[i,:] = t[i]
#     s_in_ts[i,:] = s1[samp,i]
        
# u_in_ts = (u_in_ts-min_u)/(max_u-min_u)
# x_t_in_ts = (x_t_in_ts-min_t)/(max_t-min_t)
# s_in_ts = (s_in_ts-min_s)/(max_s-min_s)

# nsamples = 100
# pred = np.zeros([nsamples,100])
# for i in range(0,nsamples):
#     pred[i,:] = np.squeeze((model({"inputsB":u_in_ts, "inputsT":x_t_in_ts})).sample(1))

In [ ]:
# ## reversing the min-max scaling
# pred = (pred*(max_s-min_s))+min_s 
# s_in_ts = (s_in_ts*(max_s-min_s))+min_s

In [ ]:
# # plt.figure(figsize = [20,10])
# plt.plot(np.arange(0,1,0.01), s_in_ts, 'b', linewidth = 2, label = 'Ground Truth')
# plt.plot(t, np.mean(pred, axis = 0),'r--', linewidth = 2, label = 'Prediction')

# plt.fill_between(t, np.mean(pred, axis = 0)+1.96*np.std(pred, axis = 0),
#                  np.mean(pred, axis = 0)-1.96*np.std(pred, axis = 0),
#                  color = 'blue', alpha = 0.25, label = '95% CI')
# plt.legend(fontsize = 20)
# plt.xlabel('t', fontsize = 20)
# plt.ylabel('$s_1$', fontsize = 20)
# plt.tight_layout()
# plt.savefig('GP_Samp_500.pdf')
# plt.show()